In [1]:
import numpy as np
import pandas as pd
import sys

path='/home/jbohn/jupyter/personal/Kernel_Learning/'
sys.path.append(path)
from Kernels.mkl_solver import primal_dual_opt
from Features.features_preproccesing import generate_features_from_quotes
from Features.features_preproccesing import generate_features_from_quotes
from Features.clean_data import clean_quotes



ModuleNotFoundError: No module named 'Features.features_preproccesing'

### Fetch Raw Quote Data

In [24]:
quotes=pd.read_csv(path+'data/AAPL_quotes.csv')
quotes

/tmp/ipykernel_3424/1904343625.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  quotes=pd.read_csv(path+'data/AAPL_quotes.csv')


,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,FINRA_BBO_Indicator,...,Best_Offer_Size,Best_Offer_FINRA_Market_Maker_ID,LULD_Indicator,LULD_NBBO_Indicator,SIP_Generated_Message_Identifier,Participant_Timestamp,FINRA_ADF_Timestamp,Security_Status_Indicator,Quote_Cancel_Correction,National_BBO_Ind
0,2020-01-02 04:00:00.065165,P,AAPL,278.00,7.0,0.00,0.0,R,2228,NaN,...,0.0,NaN,NaN,NaN,NaN,40000064785664,NaN,,NaN,2
1,2020-01-02 04:00:00.065167,P,AAPL,278.00,14.0,0.00,0.0,R,2229,NaN,...,0.0,NaN,NaN,NaN,NaN,40000064787456,NaN,,NaN,2
2,2020-01-02 04:00:00.065170,P,AAPL,293.72,9.0,0.00,0.0,R,2230,NaN,...,0.0,NaN,NaN,NaN,NaN,40000064790784,NaN,,NaN,2
3,2020-01-02 04:00:32.677788,P,AAPL,295.08,10.0,295.81,1.0,R,2676,NaN,...,1.0,NaN,NaN,NaN,NaN,40032677415424,NaN,,NaN,2
4,2020-01-02 04:00:36.260761,Q,AAPL,295.02,4.0,300.00,1.0,R,2696,NaN,...,1.0,NaN,NaN,NaN,NaN,40036260727444,NaN,,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925268,2020-01-09 19:59:47.657672,K,AAPL,310.38,20.0,310.40,7.0,R,33431302,NaN,...,151.0,NaN,NaN,NaN,NaN,195947657433000,NaN,,NaN,2
2925269,2020-01-09 19:59:48.734101,Q,AAPL,310.25,1.0,310.40,141.0,R,33431303,NaN,...,141.0,NaN,NaN,NaN,NaN,195948734083107,NaN,,NaN,2
2925270,2020-01-09 19:59:50.192454,K,AAPL,310.38,21.0,310.40,7.0,R,33431396,NaN,...,141.0,NaN,NaN,NaN,NaN,195950192198000,NaN,,NaN,2
2925271,2020-01-09 19:59:53.972452,Q,AAPL,310.25,1.0,310.40,140.0,R,33431420,NaN,...,140.0,NaN,NaN,NaN,NaN,195953972434327,NaN,,NaN,2


### Clean & Preprocessing

### Perform Normalization

In [18]:
def normalize_features(features):
    return (features-features.mean())/features.std()



features=normalize_features(features)

features,outcomes

(                          FB0       FA0       FB2       FA2
 last_interval                                              
 2020-01-02 09:30:00 -0.757752 -0.434162 -0.602106  0.062707
 2020-01-02 09:31:00 -0.757752 -0.244125 -0.053458  0.062707
 2020-01-02 09:32:00 -0.118783 -0.244125 -0.053458  0.212742
 2020-01-02 09:33:00 -0.757752 -0.244125 -0.602106  0.062707
 2020-01-02 09:34:00 -0.118783  0.135950 -0.053458  0.212742
 ...                       ...       ...       ...       ...
 2020-01-02 15:55:00  0.520187 -0.434162 -0.053458  0.062707
 2020-01-02 15:56:00  0.839672 -0.244125 -0.053458 -0.237363
 2020-01-02 15:57:00 -0.757752 -0.054088 -0.053458  0.062707
 2020-01-02 15:58:00 -0.118783  0.325987 -0.053458 -2.337850
 2020-01-02 15:59:00  0.839672 -0.244125  2.689780  0.062707
 
 [390 rows x 4 columns],
 last_interval
 2020-01-02 09:30:00   -1
 2020-01-02 09:31:00   -1
 2020-01-02 09:32:00    1
 2020-01-02 09:33:00    1
 2020-01-02 09:34:00    1
                       ..
 2020-01-

In [19]:
def batch_features(features,outcomes,batch_size):
    """ Returns a batch of features and outcomes
    """
    batched_dict={}
    for i in range(0,len(features),batch_size):
        # save the features and outcomes for each batch; timestamped by last interval 
        batched_dict[i/batch_size]={"last_interval":features.index[i] ,"features":features[i:i+batch_size], "outcomes":outcomes[i:i+batch_size]}
    return batched_dict

In [20]:
batched_dict=batch_features(features,outcomes,100)
batched_dict[0]

{'last_interval': '2020-01-02 09:30:00',
 'features':                           FB0       FA0       FB2       FA2
 last_interval                                              
 2020-01-02 09:30:00 -0.757752 -0.434162 -0.602106  0.062707
 2020-01-02 09:31:00 -0.757752 -0.244125 -0.053458  0.062707
 2020-01-02 09:32:00 -0.118783 -0.244125 -0.053458  0.212742
 2020-01-02 09:33:00 -0.757752 -0.244125 -0.602106  0.062707
 2020-01-02 09:34:00 -0.118783  0.135950 -0.053458  0.212742
 ...                       ...       ...       ...       ...
 2020-01-02 11:05:00  0.520187  0.325987 -0.053458  0.212742
 2020-01-02 11:06:00  0.520187 -0.434162  0.495190  0.062707
 2020-01-02 11:07:00 -0.438268 -0.244125 -0.602106  0.062707
 2020-01-02 11:08:00 -0.438268 -0.434162 -0.053458 -0.087328
 2020-01-02 11:09:00 -0.757752 -0.244125 -4.442640  0.062707
 
 [100 rows x 4 columns],
 'outcomes': last_interval
 2020-01-02 09:30:00   -1
 2020-01-02 09:31:00   -1
 2020-01-02 09:32:00    1
 2020-01-02 09:33:00  

### Problem Setup

### Idea is we solve the optimization problem in batches (track kernel allocation weight in live)

In [21]:
def batch_solve_mkl(X,y,m,batch_size,kernel_type,order,gap=10e-2,inner_tol=10e-3,weight_threshold=0.01,maxouter_iter=100,maxinner_iter=10 ,batch_verbose=True,verbose=True):
    """ Solves the MKL problem for a batch of data
    """
    n=X.shape[0]
    batched_dict=batch_features(X,y,batch_size)
    batched_estimates=np.zeros((n,m))
    for i in range(0,n,batch_size):
        weights,kernel=primal_dual_opt(batched_dict[i/batch_size]["features"].values,batched_dict[i/batch_size]["outcomes"].values,m,kernel_type,order,gap,inner_tol,weight_threshold,maxouter_iter,maxinner_iter ,verbose)
        batched_estimates[i,:]=weights
        if batch_verbose:
            print("Batch ",i,"Last Interval", batched_dict[i/batch_size]["last_interval"], "complete with weights ",weights)
    return batched_estimates

### Compare simulated MKL performance across each batch against single kernel performance 
- Functionality for either gaussian or polynomial basis of kernels

In [22]:
batched_estimates=batch_solve_mkl(features,outcomes,3,100,'polynomial',3,verbose=False)

Batch  0 Last Interval 2020-01-02 09:30:00 complete with weights  [0.33333333 0.33333333 0.33333333]
Batch  100 Last Interval 2020-01-02 11:10:00 complete with weights  [0.         0.33110055 0.66889945]
Batch  200 Last Interval 2020-01-02 12:50:00 complete with weights  [0.33333333 0.33333333 0.33333333]
Batch  300 Last Interval 2020-01-02 14:30:00 complete with weights  [0.33333333 0.33333333 0.33333333]
